In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.nasnet import NASNetLarge,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexByteplot"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (331,331,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(331, 331))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 331, 331, 3)

In [5]:
# Creating base_model
image_shape = (331, 331, 3)                                                                                                                                                                                                                                                                                            
base_model = NASNetLarge(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 4032)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 4032)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.1min


[CV] ........................... C=0.01, score=0.831693, total= 1.1min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.822835, total= 1.1min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.835938, total= 1.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.834661, total= 1.2min
[CV] ........................... C=0.01, score=0.831373, total= 1.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.835481, total= 1.2min
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.837302, total= 1.2min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  4.7min


[CV] ............................ C=0.1, score=0.862205, total= 4.5min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.861111, total= 4.7min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.872549, total= 4.9min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.861220, total= 4.9min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.878338, total= 4.9min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.869522, total= 5.0min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.867906, total= 5.2min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.862035, total=12.9min
[CV] ................... C=1000000000.0, score=0.849409, total=13.0min
[CV] ................... C=1000000000.0, score=0.862823, total=12.3min
[CV] ................... C=1000000000.0, score=0.866667, total=14.0min
[CV] ................... C=1000000000.0, score=0.867126, total=13.6min
[CV] ................... C=1000000000.0, score=0.868447, total=13.8min
[CV] ................... C=1000000000.0, score=0.876115, total=13.9min
[CV] ................... C=1000000000.0, score=0.855159, total=13.7min
[CV] ................... C=1000000000.0, score=0.874502, total=13.4min
[CV] .................. C=10000000000.0, score=0.860078, total=12.4min
[CV] .................. C=10000000000.0, score=0.876953, total=12.7min
[CV] .................. C=10000000000.0, score=0.872549, total=12.6min
[CV] .................. C=10000000000.0, score=0.852362, total=11.3min
[CV] .................. C=10000000000.0, score=0.871514, total= 9.4min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 67.1min remaining:  2.7min


[CV] .................. C=10000000000.0, score=0.865157, total=10.7min
[CV] .................. C=10000000000.0, score=0.859545, total= 9.9min
[CV] .................. C=10000000000.0, score=0.870775, total= 9.8min
[CV] .................. C=10000000000.0, score=0.858135, total=10.0min
[CV] .................. C=10000000000.0, score=0.879088, total=10.1min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 67.4min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 69.7658926 , 283.7893805 , 643.79766908, 750.72871335,
        824.73312488, 867.23861053, 853.23694429, 852.14034309,
        848.8505831 , 836.23222804, 844.73949506, 812.36457825,
        652.83656635]),
 'mean_score_time': array([0.24064829, 0.1997216 , 0.25819201, 0.28432593, 0.3735764 ,
        0.3465745 , 0.29671292, 0.18340979, 0.19411676, 0.2313905 ,
        0.18451283, 0.09899168, 0.05382428]),
 'mean_test_score': array([0.83602999, 0.86750197, 0.87647987, 0.86829124, 0.86621942,
        0.86335833, 0.86464088, 0.86641673, 0.86493686, 0.86444357,
        0.8660221 , 0.86612076, 0.86661405]),
 'mean_train_score': array([0.86357731, 0.9371984 , 0.99266632, 0.99901332, 0.99916694,
        0.99855279, 0.99871722, 0.99915594, 0.99905712, 0.99905714,
        0.99894741, 0.99917803, 0.99927653]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.83603, std: 0.00769, params: {'C': 0.01},
 mean: 0.86750, std: 0.00509, params: {'C': 0.1},
 mean: 0.87648, std: 0.00656, params: {'C': 1.0},
 mean: 0.86829, std: 0.00763, params: {'C': 10.0},
 mean: 0.86622, std: 0.00984, params: {'C': 100.0},
 mean: 0.86336, std: 0.00879, params: {'C': 1000.0},
 mean: 0.86464, std: 0.00681, params: {'C': 10000.0},
 mean: 0.86642, std: 0.00922, params: {'C': 100000.0},
 mean: 0.86494, std: 0.00851, params: {'C': 1000000.0},
 mean: 0.86444, std: 0.00778, params: {'C': 10000000.0},
 mean: 0.86602, std: 0.00769, params: {'C': 100000000.0},
 mean: 0.86612, std: 0.00876, params: {'C': 1000000000.0},
 mean: 0.86661, std: 0.00842, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 1.0} - score: 0.8765
